# ***Random Forest Model***

***Adding more features, all of the measures from 14 days ago***

In [41]:
import sys
sys.path.insert(1, '/Users/lauradellantonio/neuefische/Capstone/capstone')
import warnings

import pandas as pd
import numpy as np
import datetime
from datetime import timedelta

from sklearn.preprocessing import MinMaxScaler
from sklearn.compose import TransformedTargetRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics
from sklearn.model_selection import RandomizedSearchCV

import tidy_functions.load_data
import tidy_functions.clean_data
import tidy_functions.merge_data
import tidy_functions.feature_engineering

warnings.filterwarnings(action='ignore')
pd.set_option('display.max_columns', None) # To display all columns
from pprint import pprint

# Read in data

In [2]:
# Reading in survey data from csv into a dictionary of dataframes.
dfs_country = tidy_functions.load_data.load_survey_data("/Users/lauradellantonio/neuefische/Capstone/capstone/data/CMU_Global_data/Full_Survey_Data/country/smooth/", "country")

# Concatenating individuals dataframes from the dictionary into one dataframe for regions.
survey_data = pd.concat(dfs_country, ignore_index=True)

# Corona stats
covid_cases = pd.read_csv("//Users/lauradellantonio/neuefische/Capstone/capstone/data/Corona_stats/owid-covid-data.csv")
print('Read in covid data completed.')

# Mask wearing requirements
mask_wearing_requirements = pd.read_csv("/Users/lauradellantonio/neuefische/Capstone/capstone/data/data-nbhtq.csv")
print('Read in mask wearing requirements data completed.')

Read in survey data completed.
Read in covid data completed.
Read in mask wearing requirements data completed.


## Cleaning data

In [3]:
# Survey data
survey_data = tidy_functions.clean_data.delete_other_gender(survey_data)
survey_data = tidy_functions.clean_data.deal_with_NaNs_masks(survey_data)

# Corona stats
covid_cases = tidy_functions.clean_data.deal_with_NaNs_corona_stats(covid_cases)

# Mask wearing requirements
mask_wearing_requirements = tidy_functions.clean_data.prepare_mask_req(mask_wearing_requirements)
mask_wearing_requirements = tidy_functions.clean_data.dummies_mask_req(mask_wearing_requirements)
mask_wearing_requirements = tidy_functions.clean_data.dummies_public_mask_req(mask_wearing_requirements)
mask_wearing_requirements = tidy_functions.clean_data.dummies_indoors_mask_req(mask_wearing_requirements)
mask_wearing_requirements = tidy_functions.clean_data.dummies_transport_mask_req(mask_wearing_requirements)
mask_wearing_requirements = tidy_functions.clean_data.data_types_mask_req(mask_wearing_requirements)

# HDI
hdi_data = tidy_functions.clean_data.rename_hdi_countries("/Users/lauradellantonio/neuefische/Capstone/capstone/data/","hdro_statistical_data_tables_1_15_d1_d5.xlsx")
dict_hdi = tidy_functions.clean_data.create_hdi_dict(hdi_data)
dict_hdi_levels = tidy_functions.clean_data.create_hdi_levels_dict(hdi_data)

NaNs before update: 152923
NaNs after update: 0
Updated NaNs in wear_mask_all_time.
NaNs removed.
Step 1 of cleaning requirements completed.
Step 2 of cleaning requirements completed.
Step 3 of cleaning requirements completed.
Step 4 of cleaning requirements completed.
Step 5 of cleaning requirements completed.
Step 6 of cleaning requirements completed.
Creating dictionaries for hdi completed.
Creating dictionaries for hdi-levels completed.


## Merging data

In [4]:
# merging survey data with corona statistics and mask wearing requirements
covid_merge = tidy_functions.merge_data.merge_corona_stats(survey_data,covid_cases)
requirements_merge = tidy_functions.merge_data.merge_mask_req(covid_merge,mask_wearing_requirements)
hdi_merge = tidy_functions.merge_data.create_hdi_columns(requirements_merge, dict_hdi, dict_hdi_levels)

Merging corona stats completed.
Merging mask wearing requirements completed.
Creating hdi list completed.
Creating hdi-level list completed.


# Feature engineering

In [5]:
# change column to date time object and adding mask wearing requirements by date
date_fixed = tidy_functions.feature_engineering.insert_month(hdi_merge)
requirement_date = tidy_functions.feature_engineering.add_requirement_by_date(date_fixed)

Month column created.
Feature engineering completed.


In [6]:
df = requirement_date.copy()

In [7]:
# only using overall rows for both age and gender 
df = df[df["age_bucket"]=="overall"]
df = df[df["gender"]=="overall"]

In [8]:
# Columns of interest for our model
date = ["date"]

columns_general = ["iso_code", "hdi", "median_age"]

columns_general_no_iso = ["hdi", "median_age"]

columns_social_distancing = ["smoothed_pct_worked_outside_home_weighted", "smoothed_pct_grocery_outside_home_weighted", "smoothed_pct_ate_outside_home_weighted", 
                             "smoothed_pct_attended_public_event_weighted", "smoothed_pct_used_public_transit_weighted", 
                             "smoothed_pct_direct_contact_with_non_hh_weighted", "smoothed_pct_no_public_weighted"]

columns_mask_wearing = ["smoothed_pct_wear_mask_all_time_weighted", "smoothed_pct_wear_mask_most_time_weighted"]

columns_mask_req = ["cur_mask_recommended", "cur_mask_not_required", "cur_mask_not_required_recommended", "cur_mask_not_required_universal", 
                    "cur_mask_required_part_country", "cur_mask_everywhere_in_public", "cur_mask_public_indoors", "cur_mask_public_transport"]

columns_pred = ["total_cases_per_million"]

columns_interest = date + columns_general + columns_social_distancing + columns_mask_wearing + columns_mask_req + columns_pred

columns_rev_scale = columns_general_no_iso + columns_social_distancing + columns_mask_wearing + columns_mask_req + columns_pred

In [9]:
df_select = df[columns_interest]

## Total Cases per Million Past Data

In [10]:
# one day ago
previous_day = []

for i in range(len(df_select)):
    
    if df_select.at[df_select.index[i],'date'] > df_select[df_select['iso_code'] == df_select.at[df_select.index[i],'iso_code']].date.min():
        yesterday = df_select.at[df_select.index[i],'date'] - timedelta(days=1)
        iso_code = df_select.at[df_select.index[i],'iso_code']
        
        if yesterday in df_select[df_select["iso_code"] == iso_code].date.values:
            value = df_select.loc[(df_select.date == yesterday) & (df_select.iso_code == iso_code), 'total_cases_per_million']
            previous_day = [*previous_day, *value.values]
        else:
            previous_day = [*previous_day, 'NaN']
    else:
        previous_day = [*previous_day, 'NaN']
        
df_select['previous_day'] = previous_day

In [11]:
# one week ago
previous_7days = []

for i in range(len(df_select)):
    
    if df_select.at[df_select.index[i],'date'] > df_select[df_select['iso_code'] == df_select.at[df_select.index[i],'iso_code']].date.min() + timedelta(days=6):
        last_week = df_select.at[df_select.index[i],'date'] - timedelta(days=7)
        iso_code = df_select.at[df_select.index[i],'iso_code']
        
        if last_week in df_select[df_select["iso_code"] == iso_code].date.values:
            value = df_select.loc[(df_select.date == last_week) & (df_select.iso_code == iso_code), 'total_cases_per_million']
            previous_7days = [*previous_7days, *value.values]
        else:
            previous_7days = [*previous_7days, 'NaN']
    else:
        previous_7days = [*previous_7days, 'NaN']
        
df_select['previous_7days'] = previous_7days

In [12]:
# one month
previous_30days = []

for i in range(len(df_test)):
    
    if df_test.at[df_select.index[i],'date'] > df_select[df_select['iso_code'] == df_select.at[df_select.index[i],'iso_code']].date.min() + timedelta(days=29):
        last_month = df_select.at[df_select.index[i],'date'] - timedelta(days=30)
        iso_code = df_select.at[df_select.index[i],'iso_code']
        
        if last_month in df_select[df_select["iso_code"] == iso_code].date.values:
            value = df_select.loc[(df_select.date == last_month) & (df_select.iso_code == iso_code), 'total_cases_per_million']
            previous_30days = [*previous_30days, *value.values]
        else:
            previous_30days = [*previous_30days, 'NaN']
    else:
        previous_30days = [*previous_30days, 'NaN']
        
df_select['previous_30days'] = previous_30days

## Social Distancing and mask wearing past

In [78]:
def past_14_days(df, column):
    
    previous_14days = []

    for i in range(len(df)):
    
        if df.at[df.index[i],'date'] > df[df['iso_code'] == df.at[df.index[i],'iso_code']].date.min() + timedelta(days=13):
            two_weeks = df.at[df.index[i],'date'] - timedelta(days=14)
            iso_code = df.at[df.index[i],'iso_code']
        
            if two_weeks in df[df["iso_code"] == iso_code].date.values:
                value = df.loc[(df.date == two_weeks) & (df.iso_code == iso_code), column]
                previous_14days = [*previous_14days, *value.values]
            else:
                previous_14days = [*previous_14days, 'NaN']
        else:
            previous_14days = [*previous_14days, 'NaN']
    
    column_name = "14days_" + column
    df[column_name] = previous_14days
    
    return df

In [79]:
column_list = ["smoothed_pct_worked_outside_home_weighted", "smoothed_pct_grocery_outside_home_weighted", "smoothed_pct_ate_outside_home_weighted", 
               "smoothed_pct_attended_public_event_weighted", "smoothed_pct_used_public_transit_weighted", 
               "smoothed_pct_direct_contact_with_non_hh_weighted", "smoothed_pct_no_public_weighted", "smoothed_pct_wear_mask_all_time_weighted", 
               "smoothed_pct_wear_mask_most_time_weighted"]

In [80]:
for c in column_list:
    past_14_days(df_select, c)

In [83]:
columns_mask_req = ["cur_mask_recommended", "cur_mask_not_required", "cur_mask_not_required_recommended", "cur_mask_not_required_universal", 
                    "cur_mask_required_part_country", "cur_mask_everywhere_in_public", "cur_mask_public_indoors", "cur_mask_public_transport"]

In [84]:
for c in columns_mask_req:
    past_14_days(df_select, c)

## Final adjustments to dataframe

In [104]:
df_time = df_select.copy()

In [106]:
df_time.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21270 entries, 5 to 218776
Data columns (total 42 columns):
 #   Column                                                   Non-Null Count  Dtype         
---  ------                                                   --------------  -----         
 0   date                                                     21270 non-null  datetime64[ns]
 1   iso_code                                                 21270 non-null  object        
 2   hdi                                                      21270 non-null  float64       
 3   median_age                                               21270 non-null  float64       
 4   smoothed_pct_worked_outside_home_weighted                21270 non-null  float64       
 5   smoothed_pct_grocery_outside_home_weighted               21270 non-null  float64       
 6   smoothed_pct_ate_outside_home_weighted                   21270 non-null  float64       
 7   smoothed_pct_attended_public_event_weighted     

In [107]:
# Changin datatype to floats
df_time.previous_day = pd.to_numeric(df_time.previous_day, errors='coerce')
df_time.previous_7days = pd.to_numeric(df_time.previous_7days, errors='coerce')
df_time.previous_30days = pd.to_numeric(df_time.previous_30days, errors='coerce')

In [108]:
c_to_numeric = ['14days_smoothed_pct_worked_outside_home_weighted', '14days_smoothed_pct_grocery_outside_home_weighted', 
                '14days_smoothed_pct_ate_outside_home_weighted', '14days_smoothed_pct_attended_public_event_weighted', 
                '14days_smoothed_pct_used_public_transit_weighted', '14days_smoothed_pct_direct_contact_with_non_hh_weighted', 
                '14days_smoothed_pct_no_public_weighted', '14days_smoothed_pct_wear_mask_all_time_weighted', 
                '14days_smoothed_pct_wear_mask_most_time_weighted', '14days_cur_mask_recommended', '14days_cur_mask_not_required', 
                '14days_cur_mask_not_required_recommended', '14days_cur_mask_not_required_universal', '14days_cur_mask_required_part_country', 
                '14days_cur_mask_everywhere_in_public', '14days_cur_mask_public_indoors', '14days_cur_mask_public_transport']

In [109]:
for c in c_to_numeric:
    df_time[c] = pd.to_numeric(df_time[c], errors='coerce')

In [111]:
# dropping rows without history data
df_time = df_time.dropna()

In [112]:
df_time = df_time.sort_values('date')

In [113]:
df_no_iso = df_time.drop("iso_code", axis=1)
df_no_date = df_no_iso.drop("date", axis=1)

In [114]:
#divide the data into train and test data
train_size = int(len(df_no_date) * 0.80)
test_size = len(df_no_date) - train_size
train, test = df_no_date[0:train_size], df_no_date[train_size:len(df_no_date)]

In [115]:
#index the data into dependent and independent variables
train_X, train_y = train.drop("total_cases_per_million", axis=1), train["total_cases_per_million"]
test_X, test_y =  test.drop("total_cases_per_million", axis=1), test["total_cases_per_million"]
print(train_X.shape, train_y.shape, test_X.shape, test_y.shape)

(14124, 39) (14124,) (3532, 39) (3532,)


In [116]:
#to_scale = ["median_age", "smoothed_pct_worked_outside_home_weighted", "smoothed_pct_grocery_outside_home_weighted", 
#            "smoothed_pct_ate_outside_home_weighted", "smoothed_pct_attended_public_event_weighted", 
#            "smoothed_pct_used_public_transit_weighted", "smoothed_pct_direct_contact_with_non_hh_weighted", 
#            "smoothed_pct_no_public_weighted", "smoothed_pct_wear_mask_all_time_weighted", 
#            "smoothed_pct_wear_mask_most_time_weighted", "previous_day","previous_7days","previous_30days"]

In [117]:
#scale the values
#scaler_X = MinMaxScaler()
#train_X[to_scale] = scaler_X.fit_transform(train_X[to_scale])
#test_X[to_scale] = scaler_X.transform(test_X[to_scale])

In [118]:
#scaler_y = MinMaxScaler()
#scaler_y.fit(train_y)
#train_y = scaler_y.fit_transform(train_y)
#test_y = scaler_y.transform(test_y)

In [119]:
model = RandomForestRegressor(criterion='mae')

In [120]:
# define the target transform wrapper 
wrapped_model = TransformedTargetRegressor(regressor=model,transformer=MinMaxScaler()) 
# use the target transform wrapper 
wrapped_model.fit(train_X, train_y) 
yhat = wrapped_model.predict(test_X)

In [121]:
metrics.mean_absolute_error(test_y, yhat)

219.50691249292174

In [122]:
# Get the average number of nodes and the depth
n_nodes = []
max_depths = []
# Stats about the trees in random forest
for ind_tree in wrapped_model.regressor_.estimators_:
    n_nodes.append(ind_tree.tree_.node_count)
    max_depths.append(ind_tree.tree_.max_depth)
print(f'Average number of nodes:\n {int(np.mean(n_nodes))}')
print(f'Average maximum depth:\n {int(np.mean(max_depths))}')

Average number of nodes:
 16479
Average maximum depth:
 28


In [123]:
wrapped_model.regressor_.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'criterion': 'mae',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

## Grid Search

https://towardsdatascience.com/hyperparameter-tuning-the-random-forest-in-python-using-scikit-learn-28d2aa77dd74

In [124]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
pprint(random_grid)

{'bootstrap': [True, False],
 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None],
 'max_features': ['auto', 'sqrt'],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],
 'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]}


In [125]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestRegressor(criterion='mae')

# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)

# define the target transform wrapper 
wrapped_random = TransformedTargetRegressor(regressor=rf_random,transformer=MinMaxScaler()) 

# Fit the random search model
wrapped_random.fit(train_X, train_y) 

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed: 677.4min


KeyboardInterrupt: 

In [127]:
wrapped_random = TransformedTargetRegressor(regressor=rf_random,transformer=MinMaxScaler()) 

In [128]:
wrapped_random.fit(train_X, train_y) 

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


KeyboardInterrupt: 

In [ ]:
wrapped_random.regressor_.best_params_

In [ ]:
def evaluate(model, test_features, test_labels):
    predictions = model.predict(test_features)
    errors = abs(predictions - test_labels)
    mape = 100 * np.mean(errors / test_labels)
    accuracy = 100 - mape
    print('Model Performance')
    print('Average Error: {:0.4f} degrees.'.format(np.mean(errors)))
    print('Accuracy = {:0.2f}%.'.format(accuracy))
    
    return accuracy
base_model = RandomForestRegressor(n_estimators = 10, random_state = 42)
base_model.fit(train_features, train_labels)
base_accuracy = evaluate(base_model, test_features, test_labels)
Model Performance
Average Error: 3.9199 degrees.
Accuracy = 93.36%.
best_random = rf_random.best_estimator_
random_accuracy = evaluate(best_random, test_features, test_labels)
Model Performance
Average Error: 3.7152 degrees.
Accuracy = 93.73%.
print('Improvement of {:0.2f}%.'.format( 100 * (random_accuracy - base_accuracy) / base_accuracy))
Improvement of 0.40%.